In [1]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from sklearn.model_selection import train_test_split 
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
le = preprocessing.LabelEncoder()
import warnings
warnings.filterwarnings('ignore')

In [2]:
new_data=pd.read_csv("../input/csvvvv/Classeur 1 (5).csv")
new_data.drop(["Y","Word Count","Language",' '], axis=1, inplace=True)
new_data.columns = ["text", "CEFR Review","CEFR Level_Samar"]
new_data.shape

(191, 3)

In [3]:
new_data.head(2)

,text,CEFR Review,CEFR Level_Samar
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",A2,C1
1,Quoique ce détail ne touche en aucune manière ...,C1,C2


**extract samar dataset**

In [4]:
mike_20_june_samar=new_data[pd.notnull(new_data['CEFR Level_Samar'])]
mike_20_june_samar.columns = ["text", "CEFR Review","label"]
mike_20_june_samar.drop(["CEFR Review"],axis=1, inplace=True)
mike_20_june_samar.to_csv("mike_20_june_samar.csv" , index=False)
mike_20_june_samar.head()

,text,label
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",C1
1,Quoique ce détail ne touche en aucune manière ...,C2
2,"En 1804, M. Myriel était curé de Brignolles. I...",C1
3,"Vers l'époque du couronnement, une petite affa...",C2
4,—Quel est ce bonhomme qui me regarde?,C1


**extract mike_new dataset**

In [5]:
mike_20_june_new=new_data[pd.notnull(new_data['CEFR Review'])]
mike_20_june_new.columns = ["text", "label","CEFR Level_Samar"]
mike_20_june_new.drop(["CEFR Level_Samar"],axis=1, inplace=True)
mike_20_june_new.to_csv("mike_20_june_new.csv" , index=False)
mike_20_june_new.head() 

,text,label
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",A2
1,Quoique ce détail ne touche en aucune manière ...,C1
2,"En 1804, M. Myriel était curé de Brignolles. I...",A2
3,"Vers l'époque du couronnement, une petite affa...",B2
4,—Quel est ce bonhomme qui me regarde?,A2


**General insights about samar and mike_new datasets**

In [6]:
mike_20_june_samar["label"].unique(), mike_20_june_new["label"].unique()

(array(['C1', 'C2'], dtype=object),
 array(['A2', 'C1', 'B2', 'A1', 'B1', 'C2'], dtype=object))

****!!!! samar annotation seems to be weird because there is just C1 and C2****

In [7]:
mike_20_june_new.label.value_counts()

C1    58
A1    41
B2    33
B1    29
A2    24
C2     6
Name: label, dtype: int64

In [8]:
mike_20_june_samar.shape,mike_20_june_new.shape

((141, 2), (191, 2))

In [9]:
trainn=pd.read_csv("../input/frenchcefr/french dataset.csv")
trainn = trainn[trainn.label != "XX"]
trainn.head()

,text,len,lang,label
0,Il était une fois un prince qui voulait épouse...,15,FR,A2
1,Il fit le tour de la Terre pour en trouver une...,62,FR,B1
2,"Un soir, par un temps affreux, éclairs et tonn...",31,FR,B1
3,"C'était une princesse qui était là, dehors. M...",51,FR,A2
4,"« Nous allons bien voir ça », pensait la vieil...",76,FR,B1


In [10]:
lingua_fr=pd.read_csv("../input/lingua/fr_lingua.csv")

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=2400)
tfidf_data = vectorizer.fit_transform(trainn["text"]).toarray()
features = vectorizer.get_feature_names()
tfidf_dataa = pd.DataFrame(tfidf_data, columns=features)
tfidf_dataa

,1859,1866,1868,1874,abandonna,abord,abordait,absence,accable,accepta,...,étuve,été,éveilla,éveiller,éveillèrent,événements,êtes,être,êtres,œil
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.116826,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


**tfidf of the test data**

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=2400)
tfidf_data = vectorizer.fit_transform(lingua_fr["text"]).toarray()
features = vectorizer.get_feature_names()
tfidf_lingua_fr = pd.DataFrame(tfidf_data, columns=features)

In [13]:
y=trainn["label"]
y.unique()

array(['A2', 'B1', 'A1', 'B2', 'C1', 'C2'], dtype=object)

**encoding y**

In [14]:
y = le.fit_transform(y)

In [15]:
np.unique(y)

array([0, 1, 2, 3, 4, 5])

**split data**

In [16]:
xtrain, xtest, ytrain, ytest = train_test_split(tfidf_dataa, y, test_size=0.2)

****

# LGBMClassifier

In [17]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(xtrain, ytrain)
y_pred=clf.predict(xtest)
accuracy=accuracy_score(y_pred, ytest)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(ytest, y_pred)))

LightGBM Model accuracy score: 0.5556


In [18]:
np.unique(y_pred)

array([0, 1, 2, 4])

In [19]:
from sklearn.metrics import classification_report
print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.83      0.71         6
           1       0.14      0.09      0.11        11
           2       0.67      0.83      0.74        35
           3       0.00      0.00      0.00         8
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1

    accuracy                           0.56        63
   macro avg       0.24      0.29      0.26        63
weighted avg       0.46      0.56      0.50        63



In [20]:
y_true=le.transform(lingua_fr["cefr"])

In [21]:
np.unique(y_true)

array([0, 1, 2, 3])

**concatenate x train and xtest , ytrain y test and fit the model the whole data**

In [22]:
x=pd.concat([xtrain, xtest])
y_=np.concatenate([ytrain, ytest])
clf.fit(x,y_)

LGBMClassifier()

In [23]:
y_pr=clf.predict(tfidf_lingua_fr)

**how many level the model could predict**

In [24]:
np.unique(y_pr)

array([0, 1, 2])

**Classification report**

In [25]:
print(classification_report(y_true, y_pr))

              precision    recall  f1-score   support

           0       0.12      0.85      0.22        13
           1       1.00      0.04      0.07        26
           2       0.64      0.23      0.34        60
           3       0.00      0.00      0.00        12

    accuracy                           0.23       111
   macro avg       0.44      0.28      0.16       111
weighted avg       0.59      0.23      0.23       111



****

# LightGBM

In [26]:
def run_lgb(train_X, train_y, val_X, val_y,x_test):
    params = {
        "objective":"multiclass",
        "num_classes":"6",
        "metric" : "multi_error",
        "boosting_type": "gbdt",
        "num_leaves" : 10,
        "learning_rate" : 0.004,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000,
                      valid_sets=[lgtrain, lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=150, 
                      evals_result=evals_result)
    ypred=model.predict(x_test).argmax(axis=1)
    accuracy=accuracy_score(ypred, val_y)
    return  accuracy , model, evals_result

In [27]:
accuracy ,model, evals_result = run_lgb(xtrain, ytrain, xtest, ytest, xtest)
print("LightGBM Training Completed...\n\n",'LightGBM Model accuracy score: {0:0.4f}'.format(accuracy)  )

[LightGBM] [Warning] Unknown parameter: bagging_frequency
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	training's multi_error: 0.501992	valid_1's multi_error: 0.444444
LightGBM Training Completed...

 LightGBM Model accuracy score: 0.5556


**how many level the model could predict**

In [28]:
ypred=model.predict(xtest).argmax(axis=1)
np.unique(ypred)

array([2])

****

# Testing the model on lingua_fr datatset

In [29]:
ypr=model.predict(tfidf_lingua_fr).argmax(axis=1)
np.unique(ypr)

array([2])

In [30]:
print(classification_report(y_true, ypr))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.00      0.00      0.00        26
           2       0.54      1.00      0.70        60
           3       0.00      0.00      0.00        12

    accuracy                           0.54       111
   macro avg       0.14      0.25      0.18       111
weighted avg       0.29      0.54      0.38       111

